In [1]:
#直接依據我們對模型的假設，各個解釋變數間相互獨立，計算出模型參數


import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

plt.style.use("seaborn")
np.random.seed(0)

X_train_fpath = 'C:/Users/Terry/Machine_Learning-hw2/X_train'
Y_train_fpath = 'C:/Users/Terry/Machine_Learning-hw2/Y_train'
X_test_fpath = 'C:/Users/Terry/Machine_Learning-hw2/X_test'


# Parse csv files to numpy array
with open(X_train_fpath) as f:
    next(f)
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = np.float64)
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = np.float64)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = np.float64)


#feature scaling
column = np.arange(X_train.shape[1])   #510個變數
X_mean = np.mean(X_train[:, column] ,0).reshape(1, -1)
X_std  = np.std(X_train[:, column], 0).reshape(1, -1)
X_train[:,column] = (X_train[:, column] - X_mean) / (X_std + 1e-8)


In [2]:
#先將資料分作兩群
training_data= pd.DataFrame(np.concatenate((Y_train, X_train), axis=1).astype(np.float64))

training_1=np.array(training_data[training_data[0]==1].iloc[:,1:])
training_0=np.array(training_data[training_data[0]==0].iloc[:,1:])

#計算平均
mean_1=np.mean(training_1,axis=0)  #column平均
mean_0=np.mean(training_0,axis=0)  

#計算covariance
cov_0=np.zeros([len(mean_0),len(mean_0)],dtype = np.float64)
cov_1=np.zeros([len(mean_1),len(mean_1)],dtype = np.float64)

#每一個變數和總共510個變數計算離均差平方和
for x in training_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / training_0.shape[0]
for x in training_1:
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / training_1.shape[0]
    
#由於共變異數矩陣是兩個群體共用

cov= (len(training_0)*cov_0+len(training_1)*cov_1)/((len(training_1)+len(training_0)))


In [3]:
def logistic_function(z):
    
    return np.clip(1 / (1.0 + np.exp(-z)), 1e-8, 1 - (1e-8))

def predict(x_set,Parameter_set):
    
    return np.dot(x_set, Parameter_set)

def accuracy(Y_train,Y_predict):
    cross_entropy = -1*(np.dot(Y_train.transpose(), np.log(Y_predict+1e-8)) + np.dot((1 - Y_train).transpose(), np.log(1 - Y_predict+1e-8)))
    acc = 1 - np.mean(np.abs(Y_predict - Y_train))
    
    return cross_entropy,acc

In [5]:
u, s, v = np.linalg.svd(cov, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)


# Directly compute weights and bias
w = np.dot(inv,mean_0 - mean_1)
b = (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_1, np.dot(inv, mean_1))\
    + np.log(float(training_0.shape[0]) / training_1.shape[0]) 

y_predict=1-np.round(logistic_function(np.dot(X_train,w)+b))
Y_train=Y_train.reshape(len(y_predict),)
cross_entropy,acc=accuracy(Y_train,y_predict)


print('Training accuracy:', acc)
print('cross_entropy:', cross_entropy)


Training accuracy: 0.873820406959599
cross_entropy: 126107.97989899815
